# Training and hosting SageMaker Models using the Apache MXNet Module API

The **SageMaker Python SDK** makes it easy to train and deploy MXNet models. In this example, we train a simple neural network using the Apache MXNet [Module API](https://mxnet.apache.org/api/python/module/module.html) and the MNIST dataset. The MNIST dataset is widely used for handwritten digit classification, and consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits). The task at hand is to train a model using the 60,000 training images and subsequently test its classification accuracy on the 10,000 test images.

## Setup

First we need to define a few variables that will be needed later in the example.

### Install MXNet and SageMaker
_Note:  Ignore Warnings and Errors Below_

In [1]:
!pip install sagemaker --upgrade --ignore-installed --no-cache --user

    100% |████████████████████████████████| 296kB 65.5MB/s ta 0:00:01
    100% |████████████████████████████████| 133kB 63.3MB/s ta 0:00:01
    100% |████████████████████████████████| 20.2MB 95.8MB/s ta 0:00:01
    100% |████████████████████████████████| 1.3MB 85.7MB/s ta 0:00:01
    100% |████████████████████████████████| 26.1MB 98.9MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 49.5MB/s ta 0:00:01
    100% |████████████████████████████████| 71kB 70.5MB/s ta 0:00:01
    100% |████████████████████████████████| 5.9MB 98.7MB/s ta 0:00:01
    100% |████████████████████████████████| 583kB 94.4MB/s ta 0:00:01
    100% |████████████████████████████████| 133kB 84.0MB/s ta 0:00:01
    100% |████████████████████████████████| 143kB 94.2MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 67.5MB/s ta 0:00:01
    100% |████████████████████████████████| 163kB 92.5MB/s ta 0:00:01
    100% |████████████████████████████████| 235kB 90.5MB/s ta 0:00:01
    100% |███████████

## Restart the Kernel to Recognize New Dependencies Above

In [2]:
from IPython.display import display_html
display_html("<script>Jupyter.notebook.kernel.restart()</script>", raw=True)

In [1]:
!pip list

Package                            Version   
---------------------------------- ----------
absl-py                            0.9.0     
alabaster                          0.7.10    
anaconda-client                    1.6.14    
anaconda-project                   0.8.2     
asn1crypto                         0.24.0    
astor                              0.8.1     
astroid                            1.6.3     
astropy                            3.0.2     
attrs                              18.1.0    
Automat                            0.3.0     
autovizwidget                      0.13.1    
awscli                             1.16.283  
Babel                              2.5.3     
backcall                           0.1.0     
backports.shutil-get-terminal-size 1.0.0     
bcrypt                             3.1.7     
beautifulsoup4                     4.6.0     
bitarray                           0.8.1     
bkcharts                           0.2       
blaze                             

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Create the SageMaker Session

In [2]:
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

## Setup the Service Execution Role and Region
Get IAM role arn used to give training and hosting access to your data.  See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the `sagemaker.get_execution_role()` with a the appropriate full IAM role arn string(s).

In [3]:
# S3 bucket for saving code and model artifacts.
# Feel free to specify a different bucket here if you wish.
bucket = sagemaker_session.default_bucket()

# Location to save your custom code in tar.gz format.
custom_code_upload_location = 's3://{}/customcode/mxnet'.format(bucket)

# Location where results of model training are saved.
model_output_path = 's3://{}/sagemaker/mxnet-mnist/training-runs'.format(bucket)

In [4]:
role = get_execution_role()
print('RoleARN:  {}\n'.format(role))

region = sagemaker_session.boto_session.region_name
print('Region:  {}'.format(region))

RoleARN:  arn:aws:iam::362377691630:role/service-role/AmazonSageMaker-ExecutionRole-20200109T002600

Region:  us-east-1


## Training Data

The MNIST dataset has been loaded to the public S3 buckets ``sagemaker-sample-data-<REGION>`` under the prefix ``mxnet/mnist``.

In [5]:
import boto3

train_data_location = 's3://sagemaker-sample-data-{}/mxnet/mnist/train'.format(region)
test_data_location = 's3://sagemaker-sample-data-{}/mxnet/mnist/test'.format(region)

### Copy the Training Data to Your Notebook Disk

In [6]:
local_data_path = './data'

In [7]:
!aws --region {region} s3 cp --recursive {train_data_location} {local_data_path}

download: s3://sagemaker-sample-data-us-east-1/mxnet/mnist/train/labels.gz to data/labels.gz
download: s3://sagemaker-sample-data-us-east-1/mxnet/mnist/train/images.gz to data/images.gz


In [8]:
!ls {local_data_path}

images.gz  labels.gz


## Train 

### The training script

The ``mnist_mxnet.py`` script provides all the code we need for training and hosting a SageMaker model. The script also checkpoints the model at the end of every epoch and saves the model graph, params and optimizer state in the folder `/opt/ml/checkpoints`. If the folder path does not exist then it will skip checkpointing. The script we will use is adaptated from Apache MXNet [MNIST tutorial (https://mxnet.incubator.apache.org/tutorials/python/mnist.html).



In [9]:
!cat ./src/mnist_mxnet.py

import argparse
import gzip
import json
import logging
import os
import struct

import mxnet as mx
import numpy as np


def load_data(path):
    with gzip.open(find_file(path, "labels.gz")) as flbl:
        struct.unpack(">II", flbl.read(8))
        labels = np.fromstring(flbl.read(), dtype=np.int8)
    with gzip.open(find_file(path, "images.gz")) as fimg:
        _, _, rows, cols = struct.unpack(">IIII", fimg.read(16))
        images = np.fromstring(fimg.read(), dtype=np.uint8).reshape(len(labels), rows, cols)
        images = images.reshape(images.shape[0], 1, 28, 28).astype(np.float32) / 255
    return labels, images


def find_file(root_path, file_name):
    for root, dirs, files in os.walk(root_path):
        if file_name in files:
            return os.path.join(root, file_name)


def build_graph():
    data = mx.sym.var('data')
    data = mx.sym.flatten(data=data)
    fc1 = mx.sym.FullyConnected(data=data, num_hidden=128)
    act1 = mx.sym.Activation(data=fc1, act_type="relu")
 

You can add custom Python modules to the `src/requirements.txt` file.  They will automatically be installed - and made available to your training script.

In [10]:
!cat ./src/requirements.txt

# Python dependencies go here

### Train with SageMaker's `MXNet` Estimator

The SageMaker ```MXNet``` estimator allows us to run single machine or distributed training in SageMaker, using CPU or GPU-based instances.

When we create the estimator, we pass in the filename of our training script, the name of our IAM execution role, and the S3 locations we defined in the setup section. We also provide a few other parameters. ``train_instance_count`` and ``train_instance_type`` determine the number and type of SageMaker instances that will be used for the training job. The ``hyperparameters`` parameter is a ``dict`` of values that will be passed to your training script -- you can see how to access these values in the ``mnist.py`` script above.

For this example, we will choose one ``ml.m4.xlarge`` instance.

In [13]:
from sagemaker.mxnet import MXNet

mnist_estimator = MXNet(entry_point='mnist_mxnet.py',
                        source_dir='./src',
                        role=role,
                        output_path=model_output_path,
                        code_location=custom_code_upload_location,
                        train_instance_count=1,
                        train_instance_type='ml.m4.xlarge',
                        framework_version='1.4.1',
                        py_version='py3',
                        distributions={'parameter_server': {'enabled': True}},
                        hyperparameters={'learning-rate': 0.1},
                        # Assuming the logline from the MXNet training job is as follows:
                        #    Epoch[9] Train-accuracy=0.976817
                        #    Epoch[9] Time cost=1.200
                        #    Epoch[9] Validation-accuracy=0.968800                    
                        metric_definitions=[
                            {'Name':'test:accuracy', 'Regex':'Validation-accuracy=(.*?)'},
                        ])

### `fit` the Model (Approx. 15 mins)

After we've constructed our MXNet object, we can fit it using data stored in S3. Below we run SageMaker training on two input channels: **train** and **test**.

During training, SageMaker makes this data stored in S3 available in the local filesystem where the mnist script is running. The ```mnist.py``` script simply loads the train and test data from disk.

In [14]:
mnist_estimator.fit({'train': train_data_location, 'test': test_data_location}, wait=False)

training_job_name = mnist_estimator.latest_training_job.name
print('training_job_name:  {}'.format(training_job_name))

training_job_name:  mxnet-training-2020-01-09-05-43-06-038


In [ ]:
from sagemaker.mxnet import MXNet

mnist_estimator = MXNet.attach(training_job_name=training_job_name)

2020-01-09 05:43:06 Starting - Starting the training job...
2020-01-09 05:43:07 Starting - Launching requested ML instances......
2020-01-09 05:44:10 Starting - Preparing the instances for training......
2020-01-09 05:45:33 Downloading - Downloading input data
2020-01-09 05:45:33 Training - Downloading the training image.

## Option 1:  Perform Batch Predictions Directly in the Notebook

In [ ]:
print(model_output_path)

In [ ]:
!aws --region {region} s3 ls --recursive s3://sagemaker-us-east-1-835319576252/sagemaker # {model_output_path}/{training_job_name}

In [ ]:
!aws --region {region} s3 cp {model_output_path}/{training_job_name}/output/model.tar.gz ./model/model.tar.gz

In [ ]:
!ls ./model

In [ ]:
!tar -xzvf ./model/model.tar.gz -C ./model

In [ ]:
# TODO:  Perform Batch Predictions Directly in Notebook

## Option 2:  Create a SageMaker Endpoint and Perform REST-based Predictions

After training, we use the ``MXNet estimator`` object to build and deploy an ``MXNetPredictor``. This creates a Sagemaker **Endpoint** -- a hosted prediction service that we can use to perform inference. 

The arguments to the ``deploy`` function allow us to set the number and type of instances that will be used for the Endpoint. These do not need to be the same as the values we used for the training job. For example, you can train a model on a set of GPU-based instances, and then deploy the Endpoint to a fleet of CPU-based instances. Here we will deploy the model to a single ``ml.m4.xlarge`` instance.

In [ ]:
predictor = mnist_estimator.deploy(initial_instance_count=1,
                                   instance_type='ml.m4.xlarge')

The request handling behavior of the Endpoint is determined by the ``mnist_mxnet.py`` script. In this case, the script doesn't include any request handling functions, so the Endpoint will use the default handlers provided by SageMaker. These default handlers allow us to perform inference on input data encoded as a multi-dimensional JSON array.

### Making an Inference Request

Now that our Endpoint is deployed and we have a ``predictor`` object, we can use it to classify handwritten digits.

To see inference in action, draw a digit in the image box below. The pixel data from your drawing will be loaded into a ``data`` variable in this notebook. 

*Note: after drawing the image, you'll need to move to the next notebook cell.*

In [ ]:
from IPython.display import HTML
HTML(open("input.html").read())

Now we can use the ``predictor`` object to classify the handwritten digit:

In [ ]:
response = predictor.predict(data)
print('Raw prediction result:')
response = response[0]
print(response)

labeled_predictions = list(zip(range(10), response))
print('Labeled predictions: ')
print(labeled_predictions)

labeled_predictions.sort(key=lambda label_and_prob: 1.0 - label_and_prob[1])
print('Most likely answer: {}'.format(labeled_predictions[0]))

### (Optional) Delete the Endpoint

After you have finished with this example, remember to delete the prediction endpoint to release the instance(s) associated with it.

In [ ]:
print("Endpoint name: " + predictor.endpoint)

In [ ]:
import sagemaker

predictor.delete_endpoint()